# Dev Day 1

Main goals:
- Get working on toy models of superpos

Tasks:
- runner
- set 
- train loop
- wandb
- dataloader

In [44]:
from datasets import load_dataset
from transformer_lens import HookedTransformer
from sae_training.lm_datasets import preprocess_tokenized_dataset


import torch
from torch.utils.data import Dataset
from datasets import load_dataset
import einops
import os
from transformer_lens import HookedTransformer
from tqdm import tqdm

from importlib import reload
from sae_training import activations_buffer

reload(activations_buffer)

cfg = {
    "d_in": 512,
    "batch_size": 256,
    "seq_len": 128,
    "device": "cuda",
    "act_name": "blocks.0.hook_mlp_out",
    "n_batches_in_buffer": 16,
    "dtype": torch.float32,
}

model = HookedTransformer.from_pretrained("gelu-2l")
data_loader_buffer = activations_buffer.DataLoaderBuffer(
    cfg, model, data_path="NeelNanda/c4-code-tokenized-2b"
)
# batch_tokens = data_loader_buffer.get_batch_tokens()
# activations = data_loader_buffer.get_activations(batch_tokens)
# print(activations.shape)

buffer = data_loader_buffer.refill_buffer()
print(buffer.shape)

Loaded pretrained model gelu-2l into HookedTransformer


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

torch.Size([524288, 512])


In [48]:
from torch.utils.data import DataLoader
dataloader = DataLoader(buffer, batch_size=4096, shuffle=True)

while True:
    item = next(iter(dataloader))
    print(item.shape)

torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size([256, 512])
torch.Size(

generate activations:   0%|          | 0/11 [21:47<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import torch 
from functools import partial

model = HookedTransformer.from_pretrained("gelu-2l")
hook_point = "blocks.0.hook_mlp_out"
x = torch.stack(item['input_ids'],dim =1)

activations = list()
def hook_store_activation(x, hook):
    activations.append(x)
    return x 

hook_func = partial(hook_store_activation)
_ = model.run_with_hooks(
    x , fwd_hooks=
    [(hook_point, hook_func)]
)

In [33]:
torch.cuda.empty_cache()

In [19]:
batch_size = 32
seq_len = 128

batch_tokens = get_batch_tokens(
    lm=model,
    dataset=iter_dataset,
    seq_len=seq_len,
    batch_size=batch_size,
    pretokenized=True,
    device="cuda",
)

refill_buffer = False
batch_tokens.shape

torch.Size([32, 128])

In [21]:
activations = model.run_with_cache(
    batch_tokens,
    names_filter=act_name,
)[1][act_name]

print(activations.shape)

torch.Size([32, 128, 512])


In [ ]:
refill_iterator = range(0, seq_len, batch_size)
total_size = len(refill_iterator) * batch_size * seq_len + buffer.shape[0]
refill_iterator = tqdm(refill_iterator, desc="Refill")

# Initialize empty tensor buffer of the maximum required size
new_buffer = torch.zeros(total_size, *buffer.shape[1:], dtype=buffer.dtype, device=cfg["buffer_device"])

# Fill existing buffer
new_buffer[:buffer.shape[0]] = buffer

# Insert activations directly into pre-allocated buffer
insert_idx = buffer.shape[0]
for refill_batch_idx_start in refill_iterator:
    refill_batch_tokens = batch_tokens[refill_batch_idx_start:refill_batch_idx_start + cfg["batch_size"]]
    new_buffer[insert_idx:insert_idx + refill_batch_tokens.numel()] = get_activations(
        lm=lm,
        cfg=cfg,
        batch_tokens=refill_batch_tokens,
        act_name=cfg["act_name"],
    ).to(cfg["buffer_device"])
    insert_idx += refill_batch_tokens.numel()

buffer = new_buffer[:insert_idx]
buffer = buffer[torch.randperm(buffer.shape[0])]
print("... done.")

In [ ]:
if refill_buffer:
    print("Refilling buffer; time elapsed", time.time()-start_time, "...")
    if cfg["testing"] and step_idx > 100:
        raise Exception("We have finished iterating")
        # return

    # We need to refill the buffer
    # We do a similar thing when Anthropic resampling
    refill_iterator = range(0, batch_tokens.shape[0], cfg["batch_size"])
    total_size = len(refill_iterator) * cfg["batch_size"] * cfg["seq_len"] + buffer.shape[0]
    if cfg["testing"]:
        refill_iterator = tqdm(refill_iterator, desc="Refill")

    # Initialize empty tensor buffer of the maximum required size
    new_buffer = torch.zeros(total_size, *buffer.shape[1:], dtype=buffer.dtype, device=cfg["buffer_device"])

    # Fill existing buffer
    new_buffer[:buffer.shape[0]] = buffer

    # Insert activations directly into pre-allocated buffer
    insert_idx = buffer.shape[0]
    for refill_batch_idx_start in refill_iterator:
        refill_batch_tokens = batch_tokens[refill_batch_idx_start:refill_batch_idx_start + cfg["batch_size"]]
        new_buffer[insert_idx:insert_idx + refill_batch_tokens.numel()] = get_activations(
            lm=lm,
            cfg=cfg,
            batch_tokens=refill_batch_tokens,
            act_name=cfg["act_name"],
        ).to(cfg["buffer_device"])
        insert_idx += refill_batch_tokens.numel()

    # Truncate unused space in new_buffer
    buffer = new_buffer[:insert_idx]

    assert cfg["activation_training_order"] == "shuffled"
    # Shuffle buffer
    buffer = buffer[torch.randperm(buffer.shape[0])]
    print("... done.")

# Pop off the end of the buffer
mlp_post_acts = buffer[-sae_batch_size:].to(cfg["device"])
buffer = buffer[:-sae_batch_size]


In [ ]:
if refill_buffer:
print("Refilling buffer; time elapsed", time.time()-start_time, "...")
if cfg["testing"] and step_idx > 100:
    raise Exception("We have finished iterating")
    # return

# We need to refill the buffer
# We do a similar thing when Anthropic resampling
refill_iterator = range(0, batch_tokens.shape[0], cfg["batch_size"])
total_size = len(refill_iterator) * cfg["batch_size"] * cfg["seq_len"] + buffer.shape[0]
if cfg["testing"]:
    refill_iterator = tqdm(refill_iterator, desc="Refill")

# Initialize empty tensor buffer of the maximum required size
new_buffer = torch.zeros(total_size, *buffer.shape[1:], dtype=buffer.dtype, device=cfg["buffer_device"])

# Fill existing buffer
new_buffer[:buffer.shape[0]] = buffer

# Insert activations directly into pre-allocated buffer
insert_idx = buffer.shape[0]
for refill_batch_idx_start in refill_iterator:
    refill_batch_tokens = batch_tokens[refill_batch_idx_start:refill_batch_idx_start + cfg["batch_size"]]
    new_buffer[insert_idx:insert_idx + refill_batch_tokens.numel()] = get_activations(
        lm=lm,
        cfg=cfg,
        batch_tokens=refill_batch_tokens,
        act_name=cfg["act_name"],
    ).to(cfg["buffer_device"])
    insert_idx += refill_batch_tokens.numel()

# Truncate unused space in new_buffer
buffer = new_buffer[:insert_idx]

assert cfg["activation_training_order"] == "shuffled"
# Shuffle buffer
buffer = buffer[torch.randperm(buffer.shape[0])]
print("... done.")

In [7]:
act_name = "blocks.0.hook_mlp_out"
import torch
activations_list = list()
from tqdm import tqdm
for i in tqdm(range(10)):
    token_ids =  [
        torch.Tensor(next(iter(dataset))['input_ids'], dtype=Long) for _ in range(4)]
    next_input_batch = torch.stack(token_ids, dim=1)
    activations = model.run_with_cache(
        batch_tokens,
        names_filter=act_name,
    )[1][act_name]
    
    activations_list.append(activations)
    
activations = torch.cat(activations_list, dim=0)
print(activations.shape)

  0%|          | 0/10 [04:02<?, ?it/s]


NameError: name 'batch_tokens' is not defined

In [ ]:
# generate 

activations_store = list()
def hook_store_activation(x, hook):
    activations_store.append(x)
    return x

for i in range(10)

In [ ]:

start_index = self.chunk_index * self.num_batches_per_chunk * self.batch_size
end_index = start_index + self.num_batches_per_chunk * self.batch_size
chunk = data["tokens"][start_index:end_index]

# Process the chunk
chunk = einops.rearrange(chunk, "batch (x seq_len) -> (batch x) seq_len", x=8, seq_len=128)
chunk[:, 0] = self.model.tokenizer.bos_token_id
chunk = chunk[torch.randperm(chunk.shape[0])]